### This detour started because of an observation in jun rui's code:

junrui_unitary $:= e^{H_{n-1} \Delta t} ... e^{H_1 \Delta t} e^{H_0 \Delta t}$

riemann_unitary $:= e^{\sum^{k=0}_{n-1} H_k \Delta t}$

scipy_unitary $:= e^{-i \text{ * scipy.integrate.quad(H, 0, t) } }$

analytic_unitary $:= e^{-i \int_{0}^{t} H(t') dt}$

He found that riemann, scipy, and analytic unitary, all 3 of them are the same. But strangely, the junrui_unitary doesn't match the other 3 unitaries. And strangely, the junrui_unitary performs best: it can give 90+% accuracy for 4 qubit test case, while the other 3 at best gives like 60%.

We're suspecting it's because junrui_unitary is mathematically not equal to the other 3:

$$ [A, B] = 0 \implies e^A e^B = e^{A+B} $$

but the converse may not be true.

# AHHHHHH I figured it out

The math that I trust is this: According to Schrödinger eqn, it has to be true that my stepwise unitary is approximated by $\hat{U}(t, t + \Delta t) = (\mathbb{\hat{I}} - \frac{i}{\hbar} \hat{H}(t) \Delta t )$. I call this the roger_unitary. This unitary has to be correct, this is basic calculus. So this is something that I trust.

And what I found here is that junrui_unitary matches roger_unitary the most. And roger_unitary doesn't match riemann_unitary.

So conclusion: analytic_unitary is wrong... I can't just integrate H(t) elementwise then do the expm. I think I need to do do a Dyson series (I gotta figure this out later).

In [1]:
import numpy as np
from scipy.linalg import expm
import scipy
import matplotlib.pyplot as plt

In [2]:
np.set_printoptions(suppress=True)
np.set_printoptions(precision=3)

In [3]:
Values = [10, 5]
Weights = [2, 3]
Ancil = [1, 2]

alpha = 1
beta = 1
gamma = max(Values) + 1
T = 1
dt = 0.000001
t_axis = np.arange(0, T, dt)[0:]

In [4]:
sigmaI = np.array(
    [
        [1, 0],
        [0, 1],
    ]
)
sigmaX = np.array(
    [
        [0, 1],
        [1, 0],
    ]
)
sigmaBin = np.array(
    [
        [0, 0],
        [0, 1],
    ]
)

In [5]:
# H0 = np.kron(np.kron(np.kron(sigmaX, sigmaI), sigmaI), sigmaI) + np.kron(np.kron(np.kron(sigmaI, sigmaX), sigmaI), sigmaI) + np.kron(np.kron(np.kron(sigmaI, sigmaI), sigmaX), sigmaI) + np.kron(np.kron(np.kron(sigmaI, sigmaI), sigmaI), sigmaX)
H0 = np.array([[ 0.,7.14,0.,0.,0.,0.,0.,0.,0.,0.
,0.,0.,0.,0.,0.,0.,],
[ 7.14,0.,10.097,0.,0.,0.,0.,0.,0.,0.
,0.,0.,0.,0.,0.,0.,],
[ 0.,10.097,0.,12.367,0.,0.,0.,0.,0.,0.
,0.,0.,0.,0.,0.,0.,],
[ 0.,0.,12.367,0.,14.28,0.,0.,0.,0.,0.
,0.,0.,0.,0.,0.,0.,],
[ 0.,0.,0.,14.28,0.,15.966,0.,0.,0.,0.
,0.,0.,0.,0.,0.,0.,],
[ 0.,0.,0.,0.,15.966,0.,17.489,0.,0.,0.
,0.,0.,0.,0.,0.,0.,],
[ 0.,0.,0.,0.,0.,17.489,0.,18.891,0.,0.
,0.,0.,0.,0.,0.,0.,],
[ 0.,0.,0.,0.,0.,0.,18.891,0.,20.195,0.
,0.,0.,0.,0.,0.,0.,],
[ 0.,0.,0.,0.,0.,0.,0.,20.195,0.,21.42
,0.,0.,0.,0.,0.,0.,],
[ 0.,0.,0.,0.,0.,0.,0.,0.,21.42,0.
,22.579,0.,0.,0.,0.,0.,],
[ 0.,0.,0.,0.,0.,0.,0.,0.,0.,22.579
,0.,23.681,0.,0.,0.,0.,],
[ 0.,0.,0.,0.,0.,0.,0.,0.,0.,0.
,23.681,0.,24.734,0.,0.,0.,],
[ 0.,0.,0.,0.,0.,0.,0.,0.,0.,0.
,0.,24.734,0.,25.744,0.,0.,],
[ 0.,0.,0.,0.,0.,0.,0.,0.,0.,0.
,0.,0.,25.744,0.,26.715,0.,],
[ 0.,0.,0.,0.,0.,0.,0.,0.,0.,0.
,0.,0.,0.,26.715,0.,27.653],
[ 0.,0.,0.,0.,0.,0.,0.,0.,0.,0.
,0.,0.,0.,0.,27.653,0.,]])
# print(H0)

eigenvals_H0, eigenvecs_H0 = np.linalg.eigh(H0)
ground_idx_H0 = eigenvals_H0.argsort()[0]
print(eigenvals_H0[ground_idx_H0])
print(eigenvecs_H0[:, ground_idx_H0])

-47.34465334125993
[-0.     0.    -0.     0.001 -0.004  0.011 -0.026  0.056 -0.106  0.182
 -0.281  0.389 -0.475  0.5   -0.428  0.25 ]


In [6]:
Hp = (
    - (Values[0] * np.kron(np.kron(np.kron(sigmaBin, sigmaI), sigmaI), sigmaI) + Values[1] * np.kron(np.kron(np.kron(sigmaI, sigmaBin), sigmaI), sigmaI))
    + gamma * (
        (Weights[0] * np.kron(np.kron(np.kron(sigmaBin, sigmaI), sigmaI), sigmaI) + Weights[1] * np.kron(np.kron(np.kron(sigmaI, sigmaBin), sigmaI), sigmaI))
      - (Ancil[0] * np.kron(np.kron(np.kron(sigmaI, sigmaI), sigmaBin), sigmaI) + Ancil[1] * np.kron(np.kron(np.kron(sigmaI, sigmaI), sigmaI), sigmaBin))
    )**2
)
# print(Hp)

eigenvals_Hp, eigenvecs_Hp = np.linalg.eigh(Hp)
ground_idx_Hp = eigenvals_Hp.argsort()[0]
print(eigenvals_Hp[ground_idx_Hp])
print(eigenvecs_Hp[:, ground_idx_Hp])


-10.0
[0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0.]


In [7]:
def H(t):
    return alpha * ( (1 - t/T)*beta*H0 + (t/T)*Hp )

# def t_stepped(t, num_steps=3):
#     i = t // (T / num_steps)
#     return (i) * (t / num_steps)

# def H(t):
#     return alpha * ( (1 - t_stepped(t)/T)*beta*H0 + (t_stepped(t)/T)*Hp )

# print(H(0))
# print(H(T))

In [8]:
# Testing commutator relationship

In [9]:
junrui_unitary = np.identity(len(H0))

for t in t_axis:
    junrui_unitary = expm(-1j * H(t) * dt) @ junrui_unitary

print(junrui_unitary)

[[ 0.31 +0.216j  0.183-0.132j -0.423-0.263j  0.067+0.486j  0.285-0.226j
  -0.266-0.117j -0.   +0.211j  0.15 -0.059j -0.074-0.09j  -0.047+0.078j
   0.059+0.015j  0.004-0.048j -0.035+0.008j  0.015+0.02j   0.013-0.012j
  -0.017-0.001j]
 [ 0.333+0.233j  0.282+0.081j  0.315-0.274j -0.007-0.263j -0.329-0.002j
  -0.074+0.274j  0.284+0.081j  0.1  -0.22j  -0.218-0.104j -0.114+0.144j
   0.141+0.094j  0.114-0.084j -0.061-0.082j -0.09 +0.046j  0.025+0.041j
   0.073-0.033j]
 [ 0.228-0.604j -0.381+0.411j  0.215-0.143j -0.085+0.076j  0.148-0.025j
  -0.146-0.093j -0.033+0.165j  0.159-0.058j -0.07 -0.115j -0.088+0.116j
   0.099+0.032j  0.022-0.093j -0.092+0.002j  0.036+0.057j  0.056-0.019j
  -0.072-0.019j]
 [-0.126+0.044j -0.16 -0.105j -0.023-0.183j  0.04 -0.254j -0.042-0.351j
  -0.203-0.311j -0.292-0.155j -0.313+0.031j -0.196+0.205j  0.084+0.273j
   0.275+0.128j  0.179-0.104j -0.046-0.179j -0.138-0.043j -0.014+0.089j
   0.085+0.083j]
 [ 0.275+0.053j  0.257+0.165j  0.221-0.09j   0.276-0.244j  0.173-0.2

In [10]:
riemann_sum = np.zeros(H0.shape)
for t in t_axis:
    riemann_sum += H(t) * dt
riemann_unitary = expm(-1j * riemann_sum)

print(riemann_unitary)

[[ 0.668+0.642j -0.15 +0.03j   0.332+0.073j -0.044-0.012j  0.003+0.004j
   0.024-0.015j -0.004+0.003j -0.009+0.007j -0.001-0.003j  0.001-0.006j
  -0.002-0.j    -0.004+0.005j  0.   -0.j    -0.   +0.j     0.   -0.j
  -0.   +0.j   ]
 [-0.15 +0.03j   0.213+0.928j  0.223+0.134j -0.026-0.027j  0.012-0.006j
   0.017+0.003j  0.012-0.003j -0.008-0.007j -0.025-0.012j -0.011-0.003j
  -0.009-0.002j -0.006-0.002j  0.001+0.j    -0.   -0.j     0.   +0.j
   0.   -0.j   ]
 [ 0.332+0.073j  0.223+0.134j -0.782+0.404j  0.18 +0.018j  0.046-0.066j
  -0.043-0.014j  0.012-0.009j  0.017+0.j    -0.014-0.007j -0.013-0.008j
  -0.   -0.004j  0.01 +0.002j -0.001-0.j     0.   -0.j    -0.   -0.j
   0.   +0.j   ]
 [-0.044-0.012j -0.026-0.027j  0.18 +0.018j  0.57 +0.475j  0.454-0.434j
   0.081-0.094j -0.016-0.037j -0.02 -0.02j  -0.002-0.017j  0.007-0.003j
   0.001-0.006j -0.005-0.009j  0.   +0.001j -0.   -0.j     0.   +0.j
  -0.   +0.j   ]
 [ 0.003+0.004j  0.012-0.006j  0.046-0.066j  0.454-0.434j  0.462+0.579j
  -0.   

In [11]:
roger_unitary = np.identity(len(H0))

for t in t_axis:
    roger_unitary = (np.identity(len(H0)) - 1j * H(t) * dt) @ roger_unitary

print(roger_unitary)

[[ 0.31 +0.216j  0.183-0.132j -0.423-0.263j  0.067+0.486j  0.285-0.226j
  -0.266-0.117j -0.   +0.211j  0.15 -0.059j -0.074-0.09j  -0.047+0.078j
   0.059+0.015j  0.004-0.048j -0.035+0.008j  0.015+0.02j   0.013-0.012j
  -0.017-0.001j]
 [ 0.334+0.233j  0.282+0.081j  0.315-0.274j -0.007-0.263j -0.329-0.002j
  -0.074+0.274j  0.284+0.081j  0.1  -0.22j  -0.218-0.104j -0.114+0.144j
   0.141+0.094j  0.114-0.084j -0.061-0.082j -0.09 +0.046j  0.025+0.041j
   0.073-0.033j]
 [ 0.228-0.604j -0.381+0.411j  0.215-0.143j -0.085+0.076j  0.148-0.025j
  -0.146-0.093j -0.033+0.165j  0.159-0.058j -0.07 -0.115j -0.088+0.116j
   0.099+0.032j  0.022-0.093j -0.092+0.002j  0.036+0.057j  0.056-0.019j
  -0.072-0.019j]
 [-0.126+0.044j -0.161-0.105j -0.023-0.183j  0.04 -0.254j -0.042-0.352j
  -0.203-0.312j -0.292-0.155j -0.314+0.031j -0.196+0.205j  0.084+0.273j
   0.275+0.129j  0.179-0.104j -0.046-0.179j -0.139-0.043j -0.014+0.089j
   0.085+0.083j]
 [ 0.275+0.053j  0.257+0.166j  0.222-0.09j   0.277-0.245j  0.173-0.2

In [12]:
print(f"junrui - roger: {np.linalg.norm(junrui_unitary - roger_unitary)}")
print(f"junrui - riemann: {np.linalg.norm(junrui_unitary - riemann_unitary)}")
print(f"roger - riemann: {np.linalg.norm(roger_unitary - riemann_unitary)}")

junrui - roger: 0.013057111858200101
junrui - riemann: 5.647530260707473
roger - riemann: 5.650650745122526
